In [87]:
# cell 1 - imports and llm setup
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv(".env")

llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.3)

# quick test to check if connection works
response = llm.invoke("Say: I am ready to transform recipes!")
print(response.content)

It sounds like you're excited to get creative in the kitchen. What kind of transformation are you looking to make in your recipes? Do you want to:

1. Make them healthier by using more vegetables, lean proteins, or whole grains?
2. Give them a global twist by incorporating international flavors and spices?
3. Make them more vegan-friendly or gluten-free?
4. Create new desserts or sweet treats?
5. Experiment with different cuisines, such as Italian, Mexican, or Indian?

Let me know, and I'll be happy to help you get started on your culinary transformation journey!


In [88]:
# cell 2 - pydantic schema (output structure)
from pydantic import BaseModel, Field, field_validator, model_validator
from typing import List, Optional, Literal

class Ingredient(BaseModel):
    name_en: str
    name_nl: str
    quantity: Optional[str] = "to taste"

class RecipeStep(BaseModel):
    display_name_en: str = Field(description="Name of ingredient or tool, e.g. 'Onion'")
    display_name_nl: str = Field(description="Same in Dutch, e.g. 'Ui'")
    action_en: str = Field(description="Short verb phrase max 3 words, e.g. 'Wash and chop'")
    action_nl: str = Field(description="Same in Dutch, e.g. 'Wassen en snijden'")
    instructions_en: str = Field(description="Full instruction, clear for a 12-year-old")
    instructions_nl: str = Field(description="Same in Dutch")
    workplace: Literal["Stove", "Oven", "Blender", "Cutting board", "Bowl", "Pan"]
    has_timer: bool
    timer_minutes: Optional[float] = None
    is_first_appearance: bool

    @field_validator("workplace", mode="before")
    @classmethod
    def fix_workplace(cls, v):
        mapping = {
            "Plate": "Bowl",
            "Wok": "Stove",
            "Pot": "Stove",
            "Stock pot": "Stove",
            "Stock Pot": "Stove",
            "Sink": "Bowl",
            "Counter": "Cutting board",
            "Table": "Cutting board",
            "Grill": "Stove",
            "Microwave": "Oven",
            "Chopping board": "Cutting board",
            "Chopping Board": "Cutting board",
            "Toaster": "Stove",
            "Worktop": "Cutting board",
            "Kitchen counter": "Cutting board",
            "Loaf pan": "Oven",
            "Loaf Pan": "Oven",
        }
        return mapping.get(v, v)

class SousChefRecipe(BaseModel):
    generic_name_en: str
    generic_name_nl: str
    recipe_name_en: str
    recipe_name_nl: str
    description_en: str
    description_nl: str
    difficulty: Literal["Easy", "Medium", "Intermediate"]
    servings: int
    prep_time_minutes: int
    cook_time_minutes: int
    calories_per_person: Optional[int] = 0
    protein_g: Optional[float] = 0
    carbs_g: Optional[float] = 0
    fat_g: Optional[float] = 0
    ingredients: List[Ingredient]
    steps: List[RecipeStep]

    @model_validator(mode="before")
    @classmethod
    def fix_field_names(cls, v):
        renames = {
            "calories_per_serving": "calories_per_person",
            "protein_g_per_serving": "protein_g",
            "carbs_g_per_serving": "carbs_g",
            "fat_g_per_serving": "fat_g",
        }
        for old, new in renames.items():
            if old in v and new not in v:
                v[new] = v.pop(old)
        return v

    @field_validator("difficulty", mode="before")
    @classmethod
    def fix_difficulty(cls, v):
        mapping = {
            "Moderate": "Medium",
            "Hard": "Intermediate",
            "Advanced": "Intermediate",
            "Beginner": "Easy",
            "Simple": "Easy"
        }
        return mapping.get(v, v)

    @model_validator(mode="after")
    def fix_difficulty_by_steps(self):
        step_count = len(self.steps)
        if step_count <= 6:
            self.difficulty = "Easy"
        elif step_count <= 11:
            self.difficulty = "Medium"
        else:
            self.difficulty = "Intermediate"
        return self

print("schema defined ok")

schema defined ok


In [89]:
# cell 3 - langgraph agent with critique/repair loop
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# agent state - what gets passed between nodes
class AgentState(TypedDict, total=False):
    raw_recipe: str
    parsed_recipe: SousChefRecipe
    critique: str
    final_recipe: SousChefRecipe
    attempts: int

# --- prompts ---
transform_prompt = """
### Role
You are a professional recipe editor for SousChef, a cooking app for beginners.

### Task
Transform the raw recipe into SousChef format. You MUST use EXACTLY these JSON field names.

### Required JSON structure:
{{
  "generic_name_en": "Chicken Rice",
  "generic_name_nl": "Kip Rijst",
  "recipe_name_en": "One-pan Chicken and Garlic Rice",
  "recipe_name_nl": "Kip met knoflookrijst in één pan",
  "description_en": "Paragraph 1 about ease and taste. Paragraph 2 about SousChef guidance.",
  "description_nl": "Paragraaf 1. Paragraaf 2.",
  "difficulty": "Easy",
  "servings": 4,
  "prep_time_minutes": 10,
  "cook_time_minutes": 25,
  "calories_per_person": 550,
  "protein_g": 35,
  "carbs_g": 45,
  "fat_g": 18,
  "ingredients": [
    {{"name_en": "Onion", "name_nl": "Ui", "quantity": "1 piece"}}
  ],
  "steps": [
    {{
      "display_name_en": "Onion",
      "display_name_nl": "Ui",
      "action_en": "Peel and chop",
      "action_nl": "Schillen en snijden",
      "instructions_en": "Peel the onion and chop it into small pieces.",
      "instructions_nl": "Schil de ui en snijd hem in kleine stukjes.",
      "workplace": "Cutting board",
      "has_timer": false,
      "timer_minutes": null,
      "is_first_appearance": true
    }}
  ]
}}

### Rules
- workplace must be ONLY one of: "Stove", "Oven", "Blender", "Cutting board", "Bowl", "Pan"
- difficulty must be ONLY one of: "Easy", "Medium", "Intermediate"
- One micro-step per action, instructions clear for a 12-year-old
- is_first_appearance: true only on first use of that ingredient
- generic_name_en: short generic name (e.g. "Chicken Rice" not full recipe name)

### Raw Recipe
{raw_recipe}

### Output
Return ONLY valid JSON. Nothing else.
"""

critique_prompt = """
### Role
You are a strict quality checker for SousChef recipes.

### Check ALL of the following:
1. Are instructions clear enough for a 12-year-old? (no vague terms like "as needed")
2. Are timers set for ALL steps that involve waiting or cooking time?
3. Is difficulty appropriate? (Easy=few steps, Medium=some complexity, Intermediate=complex)
4. Are Dutch translations present and correct for ALL text fields?
5. Is is_first_appearance=true only on the FIRST use of each ingredient?
6. Is generic_name_en a short generic name (not the full recipe title)?
7. Are workplace values only from: Stove, Oven, Blender, Cutting board, Bowl, Pan?

### Recipe to review:
{recipe_json}

### Output
Be specific about each problem you find.
If everything is correct, write ONLY this word: APPROVED
"""

repair_prompt = """
### Role
You are a SousChef recipe editor. Fix ALL problems listed in the critique.

### Original Recipe:
{recipe_json}

### Problems to fix:
{critique}

### Rules (do not violate these):
- workplace must be ONLY: "Stove", "Oven", "Blender", "Cutting board", "Bowl", "Pan"
- difficulty must be ONLY: "Easy", "Medium", "Intermediate"
- Fix ALL mentioned problems, do not introduce new ones

### Output
Return ONLY the corrected JSON. Nothing else.
"""

# --- nodes ---
def transform_node(state: AgentState) -> AgentState:
    """converts raw recipe text into souschef format"""
    print(">> transforming recipe...")
    structured_llm = llm.with_structured_output(SousChefRecipe, method="json_mode")
    prompt = transform_prompt.format(raw_recipe=state["raw_recipe"])
    recipe = structured_llm.invoke(prompt)
    return {
        "parsed_recipe": recipe,
        "attempts": 0
    }

def critique_node(state: AgentState) -> AgentState:
    """checks recipe quality, returns APPROVED or list of problems"""
    print(">> checking recipe...")
    recipe_json = state["parsed_recipe"].model_dump_json(indent=2)
    attempts = state.get("attempts", 0)
    
    # after repair we check only critical things, otherwise it loops forever
    if attempts > 0:
        prompt = f"""
### Role
You are a quality checker for SousChef recipes. This recipe has already been repaired {attempts} time(s).

### Check ONLY critical blocking issues:
1. Are Dutch translations completely missing for any field?
2. Are any workplace values outside: Stove, Oven, Blender, Cutting board, Bowl, Pan?

### Recipe:
{recipe_json}

### Output
Only flag genuinely critical problems. Otherwise write: APPROVED
"""
    else:
        prompt = critique_prompt.format(recipe_json=recipe_json)
    
    response = llm.invoke(prompt)
    return {"critique": response.content}

def repair_node(state: AgentState) -> AgentState:
    """fixes problems from critique"""
    print(">> repairing recipe...")
    structured_llm = llm.with_structured_output(SousChefRecipe, method="json_mode")
    recipe_json = state["parsed_recipe"].model_dump_json(indent=2)
    prompt = repair_prompt.format(
        recipe_json=recipe_json,
        critique=state["critique"]
    )
    fixed_recipe = structured_llm.invoke(prompt)
    return {
        "parsed_recipe": fixed_recipe,
        "attempts": state.get("attempts", 0) + 1
    }

def finalize_node(state: AgentState) -> AgentState:
    """saves the approved recipe"""
    print(">> recipe done!")
    return {"final_recipe": state["parsed_recipe"]}

def should_repair_or_finalize(state: AgentState) -> str:
    """
    routing: if APPROVED -> finalize, if max 2 repairs reached -> finalize anyway,
    otherwise -> go to repair
    """
    critique = state.get("critique", "")
    attempts = state.get("attempts", 0)

    if "APPROVED" in critique:
        return "finalize"
    elif attempts >= 2:
        print(f"max repairs reached, finishing anyway")
        return "finalize"
    else:
        return "repair"

# --- build the graph ---
graph = StateGraph(AgentState)

graph.add_node("transform", transform_node)
graph.add_node("critique", critique_node)
graph.add_node("repair", repair_node)
graph.add_node("finalize", finalize_node)

graph.add_edge(START, "transform")
graph.add_edge("transform", "critique")
graph.add_conditional_edges("critique", should_repair_or_finalize, {
    "repair": "repair",
    "finalize": "finalize"
})
# important: repair goes BACK to critique, this is what makes the loop
graph.add_edge("repair", "critique")
graph.add_edge("finalize", END)

app = graph.compile()
print("agent ready (with critique/repair loop)")

agent ready (with critique/repair loop)


In [90]:
# cell 4 - run the agent on example recipe
raw_recipe = """
One-pan Chicken and Garlic Rice

Total Time: 35 minutes | Servings: 4

Ingredients:
1 tsp sea salt
1 tsp onion powder
1 tsp garlic powder
1 tsp sweet paprika
1 tsp dried thyme
½ tsp black pepper
¼ cup (60ml) olive oil
5 boneless chicken thighs
¼ cup (60ml) water
1 onion, finely diced
1 tsp minced garlic
1 cup (200g) jasmine rice
1½ cups (375ml) chicken stock
Fresh thyme sprigs to garnish

Instructions:
Mix salt, onion powder, garlic powder, paprika, thyme and pepper together.
Rub the spice mix all over the chicken thighs.
Heat olive oil in a large pan over medium-high heat.
Sear chicken thighs for 3-4 minutes each side until golden brown. Remove and set aside.
In the same pan, fry the diced onion for 2 minutes until soft.
Add minced garlic and fry for 1 minute.
Add rice and stir to coat in the oil.
Pour in chicken stock and water. Stir well.
Place chicken thighs on top of the rice.
Cover and cook on low heat for 20 minutes until rice is cooked and liquid absorbed.
Garnish with fresh thyme and serve.
"""

# run it
result = app.invoke({"raw_recipe": raw_recipe})

# print the result
final = result["final_recipe"]
print(f"\n{final.recipe_name_en}")
print(f"Difficulty: {final.difficulty} | Servings: {final.servings} | Cook time: {final.cook_time_minutes} min")
print(f"Calories: {final.calories_per_person} kcal | Protein: {final.protein_g}g | Carbs: {final.carbs_g}g | Fat: {final.fat_g}g")
print(f"\nDescription:\n{final.description_en}")
print(f"\nSteps ({len(final.steps)} total):")
for i, step in enumerate(final.steps, 1):
    timer = f" | {step.timer_minutes} min" if step.has_timer else ""
    print(f"  {i}. [{step.workplace}] {step.display_name_en} - {step.action_en}{timer}")
    print(f"     {step.instructions_en}")

>> transforming recipe...
>> checking recipe...
>> repairing recipe...
>> checking recipe...
>> recipe done!

One-pan Chicken and Garlic Rice
Difficulty: Medium | Servings: 4 | Cook time: 25 min
Calories: 550 kcal | Protein: 35.0g | Carbs: 45.0g | Fat: 18.0g

Description:
Cooking a delicious one-pan chicken and garlic rice is easy and quick. With SousChef, you'll be guided through each step to ensure a perfect dish every time.

Steps (9 total):
  1. [Bowl] Mix Spice Mix - Mix
     Mix salt, onion powder, garlic powder, paprika, thyme, and pepper together in a bowl.
  2. [Cutting board] Rub Chicken Thighs - Rub
     Rub the spice mix all over the chicken thighs.
  3. [Pan] Heat Olive Oil - Heat
     Heat olive oil in a large pan over medium-high heat (around 400°F/200°C).
  4. [Pan] Sear Chicken Thighs - Sear | 6.0 min
     Sear chicken thighs for 3-4 minutes each side until golden brown. Remove and set aside.
  5. [Pan] Fry Onion - Fry | 2.0 min
     Fry the diced onion for 2 minutes u

In [91]:
# cell 5 - export to json and csv
import json
import pandas as pd
import os

# create folder for outputs
os.makedirs("recipes", exist_ok=True)

final = result["final_recipe"]

# save json
output_json = final.model_dump()
with open("recipes/chicken_rice.json", "w", encoding="utf-8") as f:
    json.dump(output_json, f, indent=2, ensure_ascii=False)
print("saved: recipes/chicken_rice.json")

# save csv with steps
steps_data = []
for i, step in enumerate(final.steps, 1):
    steps_data.append({
        "step_number": i,
        "display_name_en": step.display_name_en,
        "display_name_nl": step.display_name_nl,
        "action_en": step.action_en,
        "action_nl": step.action_nl,
        "instructions_en": step.instructions_en,
        "instructions_nl": step.instructions_nl,
        "workplace": step.workplace,
        "has_timer": step.has_timer,
        "timer_minutes": step.timer_minutes,
        "is_first_appearance": step.is_first_appearance
    })

df_steps = pd.DataFrame(steps_data)
df_steps.to_csv("recipes/chicken_rice_steps.csv", index=False)
print("saved: recipes/chicken_rice_steps.csv")
print(f"\npreview:")
print(df_steps[["step_number","display_name_en","action_en","workplace","has_timer","timer_minutes"]].to_string())

saved: recipes/chicken_rice.json
saved: recipes/chicken_rice_steps.csv

preview:
   step_number        display_name_en action_en      workplace  has_timer  timer_minutes
0            1          Mix Spice Mix       Mix           Bowl      False            NaN
1            2     Rub Chicken Thighs       Rub  Cutting board      False            NaN
2            3         Heat Olive Oil      Heat            Pan      False            NaN
3            4    Sear Chicken Thighs      Sear            Pan       True            6.0
4            5              Fry Onion       Fry            Pan       True            2.0
5            6             Fry Garlic       Fry            Pan       True            1.0
6            7              Cook Rice      Cook            Pan       True            5.0
7            8             Add Liquid       Add            Pan       True           10.0
8            9  Cook Chicken and Rice      Cook            Pan       True           20.0


In [ ]:
# cell 6 - evaluation on 10 different recipes
test_recipes = {
    "scrambled_eggs": """
Scrambled Eggs
Time: 10 minutes | Servings: 2
Ingredients: 4 eggs, 2 tbsp butter, 2 tbsp milk, salt, pepper
Instructions:
Crack eggs into a bowl. Add milk, salt and pepper. Whisk well.
Melt butter in a pan over low heat.
Pour in egg mixture. Stir slowly with a spatula until just set.
Remove from heat while still slightly wet. Serve immediately.
""",
    "banana_bread": """
Banana Bread
Time: 70 minutes | Servings: 8
Ingredients: 3 ripe bananas, 80g melted butter, 150g sugar, 1 egg, 1 tsp vanilla, 1 tsp baking soda, pinch of salt, 190g flour
Instructions:
Preheat oven to 175°C. Grease a loaf pan.
Mash bananas in a bowl. Mix in melted butter.
Add sugar, egg and vanilla. Mix well.
Add baking soda and salt. Stir in flour until just combined.
Pour into loaf pan. Bake for 60 minutes until a toothpick comes out clean.
Cool for 10 minutes before slicing.
""",
    "tomato_soup": """
Tomato Soup
Time: 30 minutes | Servings: 4
Ingredients: 800g canned tomatoes, 1 onion diced, 3 garlic cloves, 2 tbsp olive oil, 500ml vegetable stock, 1 tsp sugar, salt, pepper, fresh basil
Instructions:
Heat olive oil in a pot. Fry onion for 5 minutes until soft.
Add garlic and fry 1 minute.
Add canned tomatoes, stock and sugar. Season with salt and pepper.
Simmer for 20 minutes.
Blend until smooth with an immersion blender.
Serve with fresh basil.
""",
    "pasta_bolognese": """
Pasta Bolognese
Time: 45 minutes | Servings: 4
Ingredients: 400g spaghetti, 500g minced beef, 1 onion, 2 garlic cloves, 400g canned tomatoes, 2 tbsp tomato paste, 1 tsp oregano, olive oil, salt, pepper, parmesan
Instructions:
Finely chop onion and garlic.
Heat olive oil in a pan. Fry onion 3 minutes. Add garlic 1 minute.
Add minced beef. Cook for 5 minutes until browned.
Add tomato paste, canned tomatoes and oregano. Season.
Simmer on low heat for 20 minutes.
Meanwhile cook spaghetti according to package. Drain.
Serve sauce over pasta with parmesan.
""",
    "vegetable_curry": """
Vegetable Curry
Time: 35 minutes | Servings: 4
Ingredients: 400ml coconut milk, 400g canned chickpeas, 1 onion, 2 garlic cloves, 1 tbsp ginger, 2 tbsp curry powder, 1 red pepper, 200g spinach, 2 tbsp oil, salt, rice to serve
Instructions:
Chop onion, garlic, ginger and red pepper.
Heat oil in a pan. Fry onion 3 minutes. Add garlic and ginger 1 minute.
Add curry powder and stir for 30 seconds.
Add red pepper and chickpeas. Stir well.
Pour in coconut milk. Simmer for 15 minutes.
Add spinach and stir until wilted.
Serve over rice.
""",
"chocolate_cake": """
Chocolate Cake
Time: 60 minutes | Servings: 8
Ingredients: 200g flour, 200g sugar, 50g cocoa powder, 2 tsp baking powder, 2 eggs, 200ml milk, 100ml vegetable oil, 1 tsp vanilla extract
Instructions:
Preheat oven to 180°C. Grease a round cake pan.
Mix flour, sugar, cocoa and baking powder in a bowl.
In another bowl whisk eggs, milk, oil and vanilla.
Pour wet ingredients into dry and mix until smooth.
Pour into pan and bake for 35 minutes.
Cool for 15 minutes before serving.
""",
    "greek_salad": """
Greek Salad
Time: 10 minutes | Servings: 4
Ingredients: 4 tomatoes, 1 cucumber, 1 red onion, 200g feta cheese, 100g olives, 3 tbsp olive oil, 1 tbsp oregano, salt, pepper
Instructions:
Cut tomatoes into chunks. Slice cucumber into half-moons.
Thinly slice red onion.
Combine tomatoes, cucumber and onion in a bowl.
Add olives and crumble feta on top.
Drizzle with olive oil. Season with oregano, salt and pepper.
Serve immediately.
""",
"omelette": """
Cheese Omelette
Time: 10 minutes | Servings: 1
Ingredients: 3 eggs, 30g cheddar cheese, 1 tbsp butter, salt, pepper
Instructions:
Crack eggs into a bowl. Add salt and pepper. Whisk well.
Melt butter in a pan over medium heat.
Pour in egg mixture. Let it set for 1 minute.
Sprinkle cheese on one half. Fold omelette in half.
Cook 1 more minute until cheese melts. Serve immediately.
""",
    "lentil_soup": """
Lentil Soup
Time: 40 minutes | Servings: 6
Ingredients: 300g red lentils, 1 onion, 3 garlic cloves, 2 carrots, 2 tsp cumin, 1 tsp turmeric, 1.5L vegetable stock, 2 tbsp olive oil, salt, pepper, lemon juice
Instructions:
Chop onion, garlic and carrots.
Heat olive oil in a pot. Fry onion for 5 minutes.
Add garlic, carrots, cumin and turmeric. Fry 2 minutes.
Add lentils and stock. Bring to boil.
Reduce heat and simmer for 25 minutes until lentils are soft.
Blend half the soup for a creamy texture.
Season with salt, pepper and a squeeze of lemon.
""",
    "pancakes": """
Pancakes
Time: 20 minutes | Servings: 4
Ingredients: 200g flour, 2 tsp baking powder, 1 tbsp sugar, pinch of salt, 2 eggs, 250ml milk, 2 tbsp melted butter, butter for frying
Instructions:
Mix flour, baking powder, sugar and salt in a bowl.
In another bowl whisk eggs, milk and melted butter.
Pour wet ingredients into dry. Mix until just combined, do not overmix.
Heat a pan over medium heat. Add a small knob of butter.
Pour a ladleful of batter. Cook 2 minutes until bubbles form.
Flip and cook 1 more minute. Repeat with remaining batter.
Serve with syrup or fruit.
"""
}

# run on all recipes and collect results
evaluation_results = []

for recipe_name, recipe_text in test_recipes.items():
    print(f"\n{'='*50}")
    print(f"Processing: {recipe_name}")
    try:
        result_i = app.invoke({"raw_recipe": recipe_text})
        r = result_i["final_recipe"]
        
        # save JSON
        with open(f"recipes/{recipe_name}.json", "w", encoding="utf-8") as f:
            json.dump(r.model_dump(), f, indent=2, ensure_ascii=False)
        
        evaluation_results.append({
            "recipe": recipe_name,
            "difficulty": r.difficulty,
            "servings": r.servings,
            "steps": len(r.steps),
            "ingredients": len(r.ingredients),
            "has_timers": sum(1 for s in r.steps if s.has_timer),
            "calories": r.calories_per_person,
            "status": "✅ OK"
        })
        print(f"✅ {r.recipe_name_en} | {len(r.steps)} steps | {r.difficulty}")
        
    except Exception as e:
        evaluation_results.append({
            "recipe": recipe_name, "status": f"❌ {str(e)[:50]}"
        })
        print(f"❌ Failed: {e}")

# results table
print("\n\n📊 EVALUATION RESULTS")
print("="*70)
df_eval = pd.DataFrame(evaluation_results)
print(df_eval.to_string(index=False))
df_eval.to_csv("recipes/evaluation_results.csv", index=False)
print("\n✅ Saved: recipes/evaluation_results.csv")


Processing: scrambled_eggs
>> transforming recipe...
>> checking recipe...
>> repairing recipe...
>> checking recipe...
>> recipe done!
✅ Scrambled Eggs | 7 steps | Medium

Processing: banana_bread
>> transforming recipe...
>> checking recipe...
>> recipe done!
✅ Classic Banana Bread | 9 steps | Medium

Processing: tomato_soup
>> transforming recipe...
>> checking recipe...
>> recipe done!
✅ Quick Tomato Soup | 8 steps | Medium

Processing: pasta_bolognese
>> transforming recipe...
>> checking recipe...
>> recipe done!
✅ One-pan Pasta Bolognese | 13 steps | Intermediate

Processing: vegetable_curry
>> transforming recipe...
>> checking recipe...
>> repairing recipe...
>> checking recipe...
>> recipe done!
✅ Vegetable Curry | 13 steps | Intermediate

Processing: chocolate_cake
>> transforming recipe...
>> checking recipe...
>> repairing recipe...
>> checking recipe...
>> recipe done!
✅ Chocolate Cake | 8 steps | Medium

Processing: greek_salad
>> transforming recipe...
>> checking reci

## Manual quality check

I looked at a few recipes in detail to see what works and what doesnt.

### Scrambled Eggs
**Dutch translations:** mostly ok but "Gesmolten eieren" is wrong - this literally means "melted eggs", should be "Roerei". This shows the model struggles with idiomatic Dutch expressions.
**Instructions:** clear and short, good for beginners.
**Timers:** timer on cooking step is there, but missing on the "stir slowly" step where you actually wait.
**Overall:** 6/10 - works but too generic, doesnt have that friendly SousChef tone.

### Pasta Bolognese
**Dutch translations:** acceptable. "Gehakt" is correct, "tomatenpuree" too.
**Instructions:** 14 micro-steps, each action is separate which is good.
**Timers:** set for frying and simmering, correct.
**Overall:** 7/10 - good step granularity, logical order.

### Chocolate Cake
**Dutch translations:** mostly fine, needed one repair cycle to fix some issues.
**Instructions:** "Bake until done" is too vague - SousChef style would need something like "until a toothpick comes out clean".
**Timers:** baking timer is correct.
**Overall:** 6/10 - missing visual cues that tell you when its actually done.

### Common problems I noticed
1. **Wrong workplace values** - the model keeps using things like Wok, Toaster, Plate, Pot even though I tell it not to. Fixed with validators but annoying.
2. **Literal Dutch** - translates word by word instead of how people actually say it ("Gesmolten eieren" instead of "Roerei")
3. **Vague quantities** - writes "as needed" or "to taste" in instructions instead of actual amounts
4. **Difficulty is often wrong** - critique node is too lenient, complex recipes like Chocolate Cake still get "Easy"

### Bottom line
The agent always produces valid output (10/10 passed validation), but the quality of Dutch and how specific instructions are could be much better. A bigger model would probably solve most of the translation problems.